In [27]:
import pandas as pd
import os
import regex as re
from datetime import datetime
import statsmodels.formula.api as smf
from sklearn.preprocessing import StandardScaler
from scipy.stats import pearsonr, spearmanr
from metrics import utils
import numpy as np
from lda import LDA
from gensim.parsing.preprocessing import remove_stopwords
from sklearn.feature_extraction.text import CountVectorizer
ROOT_DIR = os.path.dirname(os.path.abspath("__file__"))


In [28]:
def query_string_gen(file_dir, words):
    l=[]

    with open(file_dir, 'r') as file:
        lines = file.readlines()
    for line in lines:
        for word in words:
            if word in line:
                l.append(re.search(r'\d+', line).group())
    l = list(dict.fromkeys(l))

    string = ""
    for i,j in enumerate(l):
        if i==len(l)-1:
            string += f"Topic == {j}"
        else:
            string += f"Topic == {j} | "
    return string

# Correlation

In [29]:
def spearman_gen(dir,source,resample,pval,kind):
    events = pd.read_csv(os.path.join(ROOT_DIR,"data","Ukraine_Black_Sea_2020_2022_Dec09-1.csv"), parse_dates=["EVENT_DATE"])
    events = events[["EVENT_DATE","EVENT_TYPE","FATALITIES"]].rename(columns={"EVENT_DATE":"Date","EVENT_TYPE":"Count","FATALITIES":"Fatalities"})  # pegando somente colunas relevantes
    events = events.sort_index()
    events = events.set_index("Date") # convertendo coluna de datas pra datetime e setando indice
    events = events.loc["2022-03-01":"2022-12-09"]
    events = events.resample(resample).agg({'Count': 'count', 'Fatalities': 'sum'}) # resample: contagem de eventos, soma de fatalidades


    results = pd.read_csv(os.path.join(dir,f"{source}_Results.csv"), parse_dates=["Date"],index_col=["Date"])
    results = results[["Resonance","Novelty","Transience"]]
    results = results.sort_index()
    results = results.loc["2022-04-01":"2022-12-09"] # Matching other dataframe
    results = results.resample(resample).sum()
    results[["Count","Fatalities"]] = events[["Count","Fatalities"]].copy()

    s_fatalities, s_fatalities_p = spearmanr(results['Fatalities'], results['Resonance'])
    p_fatalities, p_fatalities_p = pearsonr(results['Fatalities'], results['Resonance'])
    s_events, s_events_p = spearmanr(results['Count'], results['Resonance'])
    p_events, p_events_p = pearsonr(results['Count'], results['Resonance'])

    if s_fatalities_p <= pval and kind == "fat":
        print(f'{source} (R X F) Spearman: {s_fatalities:.4f} p-value: {s_fatalities_p:.4f}')
    if s_events_p <= pval and kind == "eve":
        print(f'{source} (R X CE) Spearman: {s_events:.4f} p-value: {s_events_p:.4f}')

In [30]:
def pearson_gen(dir,source,resample,pval,kind):
    events = pd.read_csv(os.path.join(ROOT_DIR,"data","Ukraine_Black_Sea_2020_2022_Dec09-1.csv"), parse_dates=["EVENT_DATE"])
    events = events[["EVENT_DATE","EVENT_TYPE","FATALITIES"]].rename(columns={"EVENT_DATE":"Date","EVENT_TYPE":"Count","FATALITIES":"Fatalities"})  # pegando somente colunas relevantes
    events = events.sort_index()
    events = events.set_index("Date") # convertendo coluna de datas pra datetime e setando indice
    events = events.loc["2022-04-01":"2022-12-09"]
    events = events.resample(resample).agg({'Count': 'count', 'Fatalities': 'sum'}) # resample: contagem de eventos, soma de fatalidades


    results = pd.read_csv(os.path.join(dir,f"{source}_Results.csv"), parse_dates=["Date"],index_col=["Date"])
    results = results[["Resonance","Novelty","Transience"]]
    results = results.sort_index()
    results = results.loc["2022-04-01":"2022-12-09"] # Matching other dataframe
    results = results.resample(resample).sum()
    results[["Count","Fatalities"]] = events[["Count","Fatalities"]].copy()

    s_fatalities, s_fatalities_p = spearmanr(results['Fatalities'], results['Resonance'])
    p_fatalities, p_fatalities_p = pearsonr(results['Fatalities'], results['Resonance'])
    s_events, s_events_p = spearmanr(results['Count'], results['Resonance'])
    p_events, p_events_p = pearsonr(results['Count'], results['Resonance'])

    if p_fatalities_p <= pval and kind == "fat":
        print(f'{source} (R X F) Pearson: {p_fatalities:.4f} p-value: {p_fatalities_p:.4f}')
    if p_events_p <= pval and kind == "eve":
        print(f'{source} (R X CE) Pearson: {p_events:.4f} p-value: {p_events_p:.4f}')

In [31]:
sources = ["AP", "Fox", "CNN", "ABC", "CBS", "NYT", "Mirror", "Reuters", "Express", "HuffPost", "Guardian", "DailyMail", "All"]
variables = ["eve","fat"]
resamples = ["D", "W"]
windows = [1,3,5,7,10,15,20,25]

print("======= SPEARMAN CORRELATION =======")
for variable in variables:
    print(f"---------- Variable: {variable} ----------")
    for resample in resamples:
        print(f"----------- Resample: {resample} -----------")
        for window in windows:
            print(f"-> KLD Window: {window}")
            for source in sources:
                spearman_gen(os.path.join(ROOT_DIR,f"results_{window}"),source,resample,0.05,variable)

print("======= PEARSON CORRELATION =======")
for variable in variables:
    print(f"---------- Variable: {variable} ----------")
    for resample in resamples:
        print(f"----------- Resample: {resample} -----------")
        for window in windows:
            print(f"-> KLD Window: {window}")
            for source in sources:
                pearson_gen(os.path.join(ROOT_DIR,f"results_{window}"),source,resample,0.05,variable)

======= SPEARMAN CORRELATION =======
---------- Variable: eve ----------
----------- Resample: D -----------
-> KLD Window: 1
-> KLD Window: 3
-> KLD Window: 5
-> KLD Window: 7
All (R X CE) Spearman: -0.1468 p-value: 0.0195
-> KLD Window: 10
Express (R X CE) Spearman: -0.1574 p-value: 0.0122
All (R X CE) Spearman: -0.1435 p-value: 0.0225
-> KLD Window: 15
CBS (R X CE) Spearman: -0.1641 p-value: 0.0089
Reuters (R X CE) Spearman: -0.1693 p-value: 0.0069
All (R X CE) Spearman: -0.2191 p-value: 0.0004
-> KLD Window: 20
AP (R X CE) Spearman: -0.1242 p-value: 0.0485
CBS (R X CE) Spearman: -0.1792 p-value: 0.0042
Reuters (R X CE) Spearman: -0.1952 p-value: 0.0018
All (R X CE) Spearman: -0.2856 p-value: 0.0000
-> KLD Window: 25
CBS (R X CE) Spearman: -0.1885 p-value: 0.0026
Reuters (R X CE) Spearman: -0.2849 p-value: 0.0000
Express (R X CE) Spearman: -0.1841 p-value: 0.0033
DailyMail (R X CE) Spearman: -0.1713 p-value: 0.0063
All (R X CE) Spearman: -0.2268 p-value: 0.0003
----------- Resample:

KeyboardInterrupt: 